In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, to_timestamp, unix_timestamp
from pyspark.sql.types import DoubleType


spark = SparkSession.builder \
    .appName("US Accidents Severity Prediction") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()


25/07/20 13:21:30 WARN Utils: Your hostname, bugra-Zenbook-UX3402ZA-UX3402ZA resolves to a loopback address: 127.0.1.1; using 192.168.1.132 instead (on interface wlo1)
25/07/20 13:21:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/20 13:21:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/20 13:21:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("dataset.csv")

In [15]:
df2 = df \
    .drop("ID","Source", "Zipcode", "Timezone", "Airport_Code", "Amenity", "End_Lat", "End_Lng",
          "Bump", "Give_Way", "No_Exit", "Railway", "Description", "County", "Precipitation(in)",
          "Roundabout", "Station", "Stop", "Nautical_Twilight", "Astronomical_Twilight", "Country")
          
df2.show()

+--------+-------------------+-------------------+------------------+------------------+------------+--------------------+------------+-----+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+--------+--------+---------------+--------------+------------+--------------+--------------+
|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|Distance(mi)|              Street|        City|State|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Weather_Condition|Crossing|Junction|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Civil_Twilight|
+--------+-------------------+-------------------+------------------+------------------+------------+--------------------+------------+-----+-------------------+--------------+-------------+-----------+------------+--------------+--------------+-----------

In [ ]:
print("Weather_Condition: " + str(df2.select(col("Weather_Condition")).distinct().count()))



Weather_Condition: 145


In [22]:
# %%
# Comprehensive Column Analysis Function
def analyze_dataframe_columns(df, columns_to_analyze=None):
    """
    Comprehensive analysis of DataFrame columns
    """
    if columns_to_analyze is None:
        columns_to_analyze = df.columns
    
    print("=" * 80)
    print("COMPREHENSIVE COLUMN ANALYSIS")
    print("=" * 80)
    
    total_rows = df.count()
    print(f"Total Rows in DataFrame: {total_rows:,}")
    print("\n")
    
    # Results storage
    analysis_results = []
    
    for col_name in columns_to_analyze:
        print(f"📊 ANALYZING: {col_name}")
        print("-" * 50)
        
        try:
            # Get column data type
            col_type = dict(df.dtypes)[col_name]
            
            # Basic statistics
            distinct_count = df.select(col(col_name)).distinct().count()
            null_count = df.filter(col(col_name).isNull()).count()
            non_null_count = total_rows - null_count
            null_percentage = (null_count / total_rows) * 100
            
            print(f"Data Type: {col_type}")
            print(f"Total Values: {total_rows:,}")
            print(f"Distinct Values: {distinct_count:,}")
            print(f"Null Values: {null_count:,} ({null_percentage:.2f}%)")
            print(f"Non-Null Values: {non_null_count:,} ({100-null_percentage:.2f}%)")
            print(f"Uniqueness: {(distinct_count/non_null_count)*100:.2f}%" if non_null_count > 0 else "N/A")
            
            # Store basic info
            col_info = {
                'column': col_name,
                'type': col_type,
                'total': total_rows,
                'distinct': distinct_count,
                'nulls': null_count,
                'null_pct': null_percentage
            }
            
            # Type-specific analysis
            if col_type in ['double', 'float', 'int', 'bigint']:
                # Numerical column analysis
                print(f"📈 NUMERICAL ANALYSIS:")
                
                stats = df.select(col_name).describe().collect()
                stats_dict = {row['summary']: row[col_name] for row in stats}
                
                print(f"  Min: {stats_dict.get('min', 'N/A')}")
                print(f"  Max: {stats_dict.get('max', 'N/A')}")
                print(f"  Mean: {stats_dict.get('mean', 'N/A')}")
                print(f"  Std Dev: {stats_dict.get('stddev', 'N/A')}")
                
                # Quantiles
                if non_null_count > 0:
                    quantiles = df.select(col_name).approxQuantile(col_name, [0.25, 0.5, 0.75], 0.01)
                    if quantiles and len(quantiles) >= 3:
                        print(f"  25th Percentile: {quantiles[0]}")
                        print(f"  Median (50th): {quantiles[1]}")
                        print(f"  75th Percentile: {quantiles[2]}")
                
                col_info.update({
                    'min': stats_dict.get('min'),
                    'max': stats_dict.get('max'),
                    'mean': stats_dict.get('mean'),
                    'std': stats_dict.get('stddev')
                })
                
            else:
                # Categorical column analysis
                print(f"📋 CATEGORICAL ANALYSIS:")
                
                # Most frequent values
                top_values = df.groupBy(col_name).count().orderBy(col("count").desc()).limit(10)
                print(f"  Top 10 Most Frequent Values:")
                
                top_values_list = top_values.collect()
                for i, row in enumerate(top_values_list, 1):
                    value = row[col_name] if row[col_name] is not None else "NULL"
                    count = row['count']
                    percentage = (count / total_rows) * 100
                    print(f"    {i:2d}. '{value}': {count:,} ({percentage:.2f}%)")
                
                # Store top value info
                if top_values_list:
                    col_info['top_value'] = str(top_values_list[0][col_name])
                    col_info['top_value_count'] = top_values_list[0]['count']
                    col_info['top_value_pct'] = (top_values_list[0]['count'] / total_rows) * 100
            
            analysis_results.append(col_info)
            print("\n")
            
        except Exception as e:
            print(f"❌ Error analyzing column {col_name}: {str(e)}")
            print("\n")
    
    # Summary table
    print("=" * 80)
    print("SUMMARY TABLE")
    print("=" * 80)
    print(f"{'Column':<20} {'Type':<10} {'Distinct':<10} {'Nulls':<8} {'Null%':<8} {'Uniqueness%':<12}")
    print("-" * 80)
    
    for info in analysis_results:
        uniqueness = f"{(info['distinct']/max(info['total']-info['nulls'], 1))*100:.1f}%" if info['total']-info['nulls'] > 0 else "N/A"
        print(f"{info['column']:<20} {info['type']:<10} {info['distinct']:<10,} {info['nulls']:<8,} {info['null_pct']:<8.1f} {uniqueness:<12}")
    
    return analysis_results

# %%
# Specify your columns to analyze
columns_to_analyze = [
    "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", 
    "Wind_Direction", "Wind_Speed(mph)", "Weather_Condition", "Crossing", 
    "Junction", "Traffic_Calming", "Traffic_Signal", "Turning_Loop", 
    "Sunrise_Sunset", "Civil_Twilight"
]

# Run the analysis
print("Starting comprehensive column analysis...")
results = analyze_dataframe_columns(df2, columns_to_analyze)

# %%
# Additional specific queries you might want
print("\n" + "="*60)
print("SPECIFIC COLUMN INSIGHTS")
print("="*60)

# Boolean columns analysis
boolean_cols = ["Crossing", "Junction", "Traffic_Calming", "Traffic_Signal", "Turning_Loop"]
print("\n🔍 BOOLEAN COLUMNS DISTRIBUTION:")
for col_name in boolean_cols:
    if col_name in df2.columns:
        true_count = df2.filter(col(col_name) == True).count()
        false_count = df2.filter(col(col_name) == False).count()
        total = true_count + false_count
        if total > 0:
            print(f"{col_name}:")
            print(f"  True: {true_count:,} ({(true_count/total)*100:.1f}%)")
            print(f"  False: {false_count:,} ({(false_count/total)*100:.1f}%)")

# Weather conditions detailed analysis
print("\n🌤️ WEATHER CONDITIONS DETAILED:")
if "Weather_Condition" in df2.columns:
    weather_stats = df2.groupBy("Weather_Condition").count().orderBy(col("count").desc())
    weather_stats.show(20, truncate=False)

# Wind direction analysis
print("\n💨 WIND DIRECTION DISTRIBUTION:")
if "Wind_Direction" in df2.columns:
    wind_stats = df2.groupBy("Wind_Direction").count().orderBy(col("count").desc())
    wind_stats.show(20, truncate=False)

# %%
# Quick overview function for any column
def quick_column_overview(df, col_name):
    """Quick overview of a single column"""
    print(f"\n🔎 QUICK OVERVIEW: {col_name}")
    print("-" * 40)
    
    if col_name not in df.columns:
        print(f"❌ Column '{col_name}' not found!")
        return
    
    total = df.count()
    distinct = df.select(col_name).distinct().count()
    nulls = df.filter(col(col_name).isNull()).count()
    
    print(f"Total rows: {total:,}")
    print(f"Distinct values: {distinct:,}")
    print(f"Null values: {nulls:,} ({(nulls/total)*100:.1f}%)")
    print(f"Data type: {dict(df.dtypes)[col_name]}")
    
    # Show sample values
    print("Sample values:")
    df.select(col_name).distinct().limit(5).show(truncate=False)

# Example usage:
# quick_column_overview(df2, "Weather_Condition")
# quick_column_overview(df2, "Wind_Speed(mph)")

print("\n✅ Analysis completed! Use quick_column_overview(df2, 'column_name') for individual column details.")

columns_to_analyze = [
    "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", 
    "Wind_Direction", "Wind_Speed(mph)", "Weather_Condition", "Crossing", 
    "Junction", "Traffic_Signal", "Sunrise_Sunset", "Civil_Twilight"
]

# Analizi çalıştır
print("Analiz başlıyor...")
results = analyze_dataframe_columns(df2, columns_to_analyze)

Starting comprehensive column analysis...
COMPREHENSIVE COLUMN ANALYSIS


Total Rows in DataFrame: 7,728,394


📊 ANALYZING: Wind_Chill(F)
--------------------------------------------------


Data Type: double
Total Values: 7,728,394
Distinct Values: 1,002
Null Values: 1,999,019 (25.87%)
Non-Null Values: 5,729,375 (74.13%)
Uniqueness: 0.02%
📈 NUMERICAL ANALYSIS:


  Min: -89.0
  Max: 207.0
  Mean: 58.25104839533092
  Std Dev: 22.3898317496207


  25th Percentile: 43.0
  Median (50th): 62.0
  75th Percentile: 75.0


📊 ANALYZING: Humidity(%)
--------------------------------------------------


...

Data Type: string
Total Values: 7,728,394
Distinct Values: 3
Null Values: 23,246 (0.30%)
Non-Null Values: 7,705,148 (99.70%)
Uniqueness: 0.00%
📋 CATEGORICAL ANALYSIS:
  Top 10 Most Frequent Values:


     1. 'Day': 5,334,553 (69.03%)
     2. 'Night': 2,370,595 (30.67%)
     3. 'NULL': 23,246 (0.30%)


📊 ANALYZING: Civil_Twilight
--------------------------------------------------


Data Type: string
Total Values: 7,728,394
Distinct Values: 3
Null Values: 23,246 (0.30%)
Non-Null Values: 7,705,148 (99.70%)
Uniqueness: 0.00%
📋 CATEGORICAL ANALYSIS:
  Top 10 Most Frequent Values:


     1. 'Day': 5,695,619 (73.70%)
     2. 'Night': 2,009,529 (26.00%)
     3. 'NULL': 23,246 (0.30%)


SUMMARY TABLE
Column               Type       Distinct   Nulls    Null%    Uniqueness% 
--------------------------------------------------------------------------------


TypeError: max() takes 1 positional argument but 2 were given